In [1]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()
"""
Issues Encountered:
cant have negative numbers in multidimensional arrays
cant have camel case words
cant have float attributes
"""

Note: you may need to restart the kernel to use updated packages.


'\nIssues Encountered:\ncant have negative numbers in multidimensional arrays\ncant have camel case words\ncant have float attributes\n'

In [2]:
### Set up Yolo UDF
cursor.query("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
    """).df() 

,0
0,"UDF Yolo already exists, nothing added."


In [3]:
### Set up Monodepth UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS MonodepthDetection
            IMPL'/home/youse/apperception/eva/udfs/monodepth_detection.py';
    """).df()

,0
0,"UDF MonodepthDetection already exists, nothing..."


In [4]:
### Set up Location UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS LocationDetection
            IMPL'/home/youse/apperception/eva/udfs/location_detection.py';
    """).df()

,0
0,"UDF LocationDetection already exists, nothing ..."


In [5]:
### Set up Intersection Query UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS IntersectionQuery
            IMPL'/home/youse/apperception/eva/udfs/intersection_query.py';
    """).df()

,0
0,"UDF IntersectionQuery already exists, nothing ..."


In [6]:
### Load video
video_name = "boston-seaport-scene-0757-CAM_FRONT.mp4"
scene = "scene-0757"
video_path = "/data/processed/full-dataset/mini/videos/"
cursor.query("DROP TABLE IF EXISTS ObjectDetectionVideos;").df()
cursor.load(file_regex=video_path + video_name, format="VIDEO", table_name='ObjectDetectionVideos').df()

,0
0,Number of loaded VIDEO: 1


In [7]:
cursor.query("DROP TABLE IF EXISTS CameraConfigs;").df()
cursor.create_table("CameraConfigs", if_not_exists=True, columns="""
                cameraid TEXT(15),
                framenum INTEGER,
                cameratranslation NDARRAY FLOAT32(ANYDIM),
                camerarotation TEXT(100),
                cameraintrinsic NDARRAY FLOAT32(ANYDIM),
                egoheading INTEGER
            """).df()


""


In [8]:
### Add Camera Configs
%cd ..
from apperception.database import database
import pandas as pd


result = database.execute(f"SELECT cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading FROM Cameras WHERE cameraId = '{scene}'")
df = pd.DataFrame()
for r in result:
    cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading = r
    cameraTranslation = list(cameraTranslation)
    cursor.query(f"""INSERT INTO CameraConfigs (cameraid, framenum, cameratranslation, camerarotation, cameraintrinsic, egoheading) VALUES
                                ('{cameraId}', {frameNum}, {cameraTranslation}, '{cameraRotation}', {cameraIntrinsic}, {egoHeading});""").df()
df
    

/home/youse/apperception


""


In [9]:
cursor.query("SELECT id, egoheading FROM ObjectDetectionVideos JOIN CameraConfigs ON id = framenum").df()

,objectdetectionvideos.id,cameraconfigs.egoheading
0,1,63.418377
1,2,63.410492
2,3,63.406160
3,4,63.397694
4,5,63.397503
5,6,63.394688
6,7,63.392323
7,8,63.392437
8,9,63.391420


In [10]:
cursor.query("SELECT MonodepthDetection(data).depth FROM ObjectDetectionVideos JOIN CameraConfigs ON id = framenum limit 1").df()

/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


CUDA is available.
 > 0: Tesla T4


/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


mondepth


,monodepthdetection.depth
0,"[[[12.378500938415527, 12.38070297241211, 12.3..."


In [11]:
### Run query
a=cursor.query("""
            SELECT LocationDetection(Yolo(data).bboxes, MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic) FROM ObjectDetectionVideos JOIN CameraConfigs ON id = framenum
""").df()
a


/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


mondepth
[   -0.58233    -0.37711    -0.61587    -0.37334] <class 'numpy.ndarray'>
[   -0.58236    -0.37701    -0.61589    -0.37336] <class 'numpy.ndarray'>
[   -0.58237    -0.37696    -0.61591    -0.37336] <class 'numpy.ndarray'>
[   -0.58247    -0.37664    -0.61587     -0.3736] <class 'numpy.ndarray'>
[   -0.58245    -0.37662    -0.61589    -0.37363] <class 'numpy.ndarray'>
[   -0.58249    -0.37658    -0.61586    -0.37364] <class 'numpy.ndarray'>
[   -0.58251    -0.37658    -0.61586    -0.37361] <class 'numpy.ndarray'>
[   -0.58255    -0.37658    -0.61583    -0.37361] <class 'numpy.ndarray'>
[   -0.58262    -0.37656    -0.61577    -0.37362] <class 'numpy.ndarray'>


,locationdetection.locations
0,"[[1254.342529296875, 425.23553466796875, 1471...."
1,"[[1262.8585205078125, 426.02020263671875, 1470..."
2,"[[1277.282958984375, 429.562744140625, 1503.24..."
3,"[[1275.486083984375, 423.3941650390625, 1532.6..."
4,"[[1283.18603515625, 425.26666259765625, 1553.6..."
5,"[[1199.7728271484375, 356.6239929199219, 1219...."
6,"[[1199.3699951171875, 353.643798828125, 1219.9..."
7,"[[1199.689208984375, 352.7824401855469, 1220.3..."
8,"[[1199.8338623046875, 348.64501953125, 1222.21..."
